---

# Flamespeed

---

## What is Flamespeed?

Flamespeed is an open-source Python library with object-orientated methods for chemical kinetics with the following core functionality:

* API to import a database of elementary reactions in XML-format
* Efficiently calculate reaction rates for systems of irreversible elementary chemical reactions
* Output provided in format suitable for further analysis

Potential **applications** for Flamespeed includes:

* Validation of chemical phase input files
* Input for solving ODEs for system of reactions
* Input for machine learning methods (e.g. neural networks) that predict reaction types

## Installation

There are two ways to install Flamespeed:

**Install Flamespeed from PyPI (recommended)**

```bash
pip install flamespeed
```

**Alternatively: Install Flamespeed form Github source**

First, clone Flamespeed using `git`:

```sh
git clone https://github.com/bkleyn/flamespeed.git
```
Then, `cd` to the root folder and run the install command:
```sh
python setup.py install
```

The package **test suite** can be run as follows:

First, `cd` to the flamespeed folder and then run pytest:
```sh
cd flamespeed
pytest
```

**Requirements:**
* Python (>= 3.3)
* NumPy (>= 1.8.2)
* SciPy (>= 0.13.3)


## Getting started

Import the `chemkin` module in flamespeed, which is the main module to evaluate chemical reaction rates.

```python
from flamespeed import chemkin
```
Instantiate the reaction rates class and import reaction database in XML format. Example reaction files can be found in the `./data` directory within `flamespeed`.

```python
a = chemkin.ReactionRate()
a.read_XML('./flamespeed/data/rxns_reversible.xml')
print(a)
```
Next, set the initial concentrations for each of the species included in the reaction and the temperature.

```python
a.set_temp(750)
x = [2.0, 1.0, 0.5, 1.0, 1.0, 1.5, 0.5, 1]
```
Now that all the input parameters have been specified the reaction rates for each of the species in the system can be determined as follows:

```python
r = a.get_reaction_rate(x)
print(r)
```

---

## Technical detail

#### Irreversible Elementary Reactions

The mathematical and physical underpinnings of this chemical kinetics library is to solve for the reaction rates of a system consisting of $N$ species undergoing $M$ **irreversible**, **elementary** reactions of the form:

\begin{align}
  \sum_{i=1}^{N}{\nu_{ij}^{\prime}\mathcal{S}_{i}} \longrightarrow 
  \sum_{i=1}^{N}{\nu_{ij}^{\prime\prime}\mathcal{S}_{i}}, \qquad j = 1, \ldots, M
\end{align}

An example system of reactions for which the reaction rates can be determined by this library is:

$$ H_{2} + O_{2} \longrightarrow H_{2}O + O$$
$$ H_{2}O + O \longrightarrow 2OH $$
$$ OH + H_{2} \longrightarrow 2H + OH $$

More specifically, the **reaction rate** of specie $i$ can be written as:

\begin{align}
  f_{i} = \sum_{j=1}^{M}{\nu_{ij}\omega_{j}}, \qquad i = 1, \ldots, N
\end{align}

where the **progress rate** for each reaction is denoted by:

\begin{align}
  \omega_{j} = k_{j}\prod_{i=1}^{N}{x_{i}^{\nu_{ij}^{\prime}}}, \qquad j = 1, \ldots, M
\end{align}

Please refer to the table below for an overview of used symbols. Note that $k_{j}$ is the forward reaction rate coefficient, and is a direct input to a reaction's progress rate. Specifically, this library is capable of handling and calculating three different types of reaction rate coefficients:

$$ Constant: k_j = c $$
$$ Arrhenius: k_j = Ae^{(-\dfrac{E}{RT})}$$
$$ Modified \hspace{0.25em} Arrhenius: k_j = AT^{b}e^{(-\dfrac{E}{RT})}$$

The library assigns each elementary reaction its own reaction rate coefficient. When loading an XML-file with data, the rate coefficient type needs to be specified. In case of a Arrhenius reaction rate coefficient, additional values need to be provided for $A$, $E$, $R$, $T$. In case of Modified Arrhenius reaction rate coefficient, additional values need to be provided for $A$, $b$, $E$, $R$, $T$.

---

#### Reversible Elementary Reactions

Although the above derivations hold for **irreversible** elementary reactions, oftentimes **reversible** elementary reactions occur in nature instead (i.e., the products can react and produce the reactants and so forth). Generally, a reversible reaction will have the form:

$$\sum_{i=1}^{N}{\nu_{ij}^{\prime}\mathcal{S}_{i}}  \rightleftharpoons \sum_{i=1}^{N}{\nu_{ij}^{\prime\prime}\mathcal{S}_{i}} \qquad j = 1, \ldots, M$$

where $\rightleftharpoons$ indicts that the foward and backward reactions occur. Note this sign is technically the only difference with the above derive reaction forms.

The total progress rate is then given by:

$$\omega_{j} = k_{j}^{\left(f\right)}\prod_{i=1}^{N}{x_{i}^{\nu_{ij}^{\prime}}} - k_{j}^{\left(b\right)}\prod_{i=1}^{N}{x_{i}^{\nu_{ij}^{\prime\prime}}}, \qquad j = 1,\ldots, M.$$

If, and only if, the reaction is elementary, the **backward reaction coefficient** can be defined as:

$$k_{j}^{\left(b\right)} = \frac{k_{j}^{\left(f\right)}}{k_{j}^{e}}, \qquad j =1, \ldots, M$$

where $k_{j}^{e}$ is the **equilibrium coefficient** for reaction $j$.

The final expression for the equilibrium coefficient is

$$k_{j}^{e} = \left(\frac{p_{0}}{RT}\right)^{\gamma_{j}}\exp\left(\frac{\Delta S_{j}}{R} - \frac{\Delta H_{j}}{RT}\right), \qquad j =1, \ldots, M$$

where $\gamma_{j} = \sum_{i=1}^{N}{\nu_{ij}}$ and $p_{0}$ is the pressure of the reactor (take it to be $10^{5}$ Pa).

The entropy and enthalpy change of reaction $j$ can be defined in terms of the entropy and enthalpy of the species participating in reaction $j$.  We have:

$$\Delta S_{j} = \sum_{i=1}^{N}{\nu_{ij}S_{i}} \quad \textrm{and} \quad \Delta H_{j} = \sum_{i=1}^{N}{\nu_{ij}H_{i}}, , \qquad j =1, \ldots, M.$$

The **specific heat** at constant pressure ($C_{p,i}$), **enthalpy** ($H_{i}$) and **entropy** ($S_{i}$) are related via

$$H_{i} = \int_{T_{0}}^{T}{C_{p,i}\left(T\right) \ \mathrm{d}T}, \qquad i = 1, \ldots, N$$

and 

$$S_{i} = \int_{T_{0}}^{T}{\frac{C_{p,i}\left(T\right)}{T} \ \mathrm{d}T}, \qquad i = 1, \ldots, N$$

where $C_{p,i}$ is given by a polynomial in $T$ (called the **NASA polynomial**), 

$$C_{p,i} = \left(\sum_{k=1}^{5}{a_{ik}T^{k-1}}\right)R, \qquad i = 1, \ldots, N.$$

The $7$th order NASA polynomials are given by 

$$\frac{C_{p,i}}{R} = a_{i1} + a_{i2}T + a_{i3}T^{2} + a_{i4}T^{3} + a_{i5}T^{4}$$

$$\frac{H_{i}}{RT} = a_{i1} + \frac{1}{2}a_{i2}T + \frac{1}{3}a_{i3}T^{2} + \frac{1}{4}a_{i4}T^{3} + \frac{1}{5}a_{i5}T^{4} + \frac{a_{i6}}{T}$$

$$\frac{S_{i}}{R} = a_{i1}\ln\left(T\right) + a_{i2}T + \frac{1}{2}a_{i3}T^{2} + \frac{1}{3}a_{i4}T^{3} + \frac{1}{4}a_{i5}T^{4} + a_{i7}$$

for $i = 1,\dots, N$.

---

A few notes on symbols used:

| Symbol | Meaning |
|:--------:|:-------:|
| $\mathcal{S}_{i}$ | Chemical symbol of specie $i$ |
| $\nu_{ij}^{\prime}$ | Stoichiometric coefficients of reactants |
| $\nu_{ij}^{\prime\prime}$ | Stoichiometric coefficients of products |
| $N$                       | Number of species in system |
| $M$                       | Number of elementary reactions |
| $R$                       | Ideal gas constant |
| $T$                       | Temperature of reaction |
| $S_i$                     | Entropy of specie $i$ |
| $H_i$                     | Enthalpy of specie $i$ |
| $C_{p,i}$                 | Specific heat at constant pressure of specie $i$ |
| $f_{i}$                   | Rate of consumption or formation of specie $i$ (reaction rate) |
| $\omega_{j}$              | Progress rate of reaction $j$ |
| $x_{i}$                   | Concentration of specie $i$ |
| $k^f_{j}$                 | (Forward) Reaction rate coefficient for reaction $j$ |
| $k^b_{j}$                 | Backward reaction rate coefficient for reaction $j$ |
| $k^e_{j}$                 | Equilibrium reaction rate coefficient for reaction $j$ |
| $a_{i}$                   | NASA polynomial coefficients |

---


## Chemkin module

### ReactionCoefficients Class

**Methods:**
    1. k_const(k=1.0) - Simply returns a constant reaction rate coefficient
    2. k_arrhenius(A, E, T, R=8.314) - Arrhenius reaction rate coefficient
    3. k_arrhenius_mod(A, b, E, T, R=8.314) - Modified Arrhenius reaction rate coefficient

**k_const(k=1.0)**
    
    Parameters:
    k: float, default value = 1.0 - Constant reaction rate coefficient

    Returns:
    k: Constant reaction rate coefficient

**k_arrhenius(A, E, T, R=8.314)**
    
    Parameters:
    A: float, positive - Arrhenius prefactor
    E: float - Activation energy
    T: float, positive - Temperature in Kelvin
    R: float, positive, default value = 8.314 - Ideal gas constant
    
    Returns:
    k: Arrhenius reaction rate coefficient

**k_arrhenius_mod(A, b, E, T, R=8.314)**
    
    Parameters:
    A: float, positive - Arrhenius prefactor
    b: float - Modified Arrhenius parameter
    E: float - Activation energy
    T: float, positive - Temperature in Kelvin
    R: float, positive, default value = 8.314 - Ideal gas constant
    
    Returns:
    k: Modified Arrhenius reaction rate coefficient

### ReactionRate Class

**Methods:**
    1. __init__ - Initialize ReactionRate class
    2. __str__ - Return summary of XML file contents
    3. __repr__ - Return class type
    4. __len__ - Returns number of reactions
    5. read_XML(path) - Reads XML from file path and extracts reaction information
    6. set_temp(T) - Calculate reaction rate coefficients at specified temperature
    7. get_progress_rate(concs) - Progress rate for a system of elementary reactions
    8. get_reaction_rate(concs) - Reaction rate for a system of elementary reactions

**\__init__**
    
    Sets class-wide parameters

**\__str__**
    
    Returns: Returns summary of reaction phase, including species and number of reactions

**\__repr__**
    
    Returns: Class type

**\__len__**
    
    Returns: Number of reactions

**read_XML(path)**

    Parameters:
    path: string, path to the XML file
    
    Returns:
    ReactionRate class instance

**set_temp(T)**

    Parameters:
    T: Float, positive - Temperature in Kelvin
    
    Returns:
    k: float - Reaction rate coefficients for each reaction

**get_progress_rate(concs)**

    Parameters:
    concs: numpy array of floats (size: num_species) - Concentration of species
    
    Returns:
    w: numpy array of floats (size: num_reactions) - Progress rate for each reaction

**get_progress_rate(concs)**

    Parameters:
    concs: numpy array of floats (size: num_species) - Concentration of species
    
    Returns:
    f: numpy array of floats (size: num_species) - Reaction rate for each specie

---

## Thermo module

**Methods:**

    1. __init__(species, nu_coef) - Initialize ReactionRate class
    2. read_nasa_coeffs(T) - Read thermo.sqlite database to obtain NASA coefficients
    3. Cp_over_R(T) - Computes specific heat over ideal gas constant (R) at specified temperature
    4. H_over_RT(T) - Computes enthalpy change (delta H) over (R*T) at specified temperature
    5. S_over_R(T) - Computes entropy change (delta S) over (R) at specified temperature
    6. backward_coeffs(kf, T) - Calculate the backward coefficents given forward rate coefficients
    
**\__init__**
    
    Sets class-wide parameters
    
**read_nasa_coeffs(T)**

    Parameters:
    T: Temperature (in Kelvin)
    
    Returns:
    NASA Coefficients
    
**Cp_over_R(T)**

    Parameters:
    T: Temperature (in Kelvin)
    
    Returns:
    Specific Heat at constant pressure over Ideal Gas Constant (C/R)
    
**H_over_RT(T)**

    Parameters:
    T: Temperature (in Kelvin)
    
    Returns:
    Enthalpy Change (Delta S) over Ideal Gas Constant times Temperature (RT)
    
**S_over_R(T)**

    Parameters:
    T: Temperature (in Kelvin)
    
    Returns:
    Entropy Change (Delta S) over Ideal Gas Constant (R)
    
**S_over_R(T)**

    Parameters:
    kf: Forward Reaction Rate Coefficient
    T: Temperature (in Kelvin)
    
    Returns:
    Backward coefficient kf / ke where ke is the equilibrium coefficient

---

## Parser module

### ReactionParser Class

**Methods:**

    1.  __init__(supported, hook) - Initialize ReactionRate class
    2.  parse_species(rxns) - Return list of species in reaction data
    3.  check_keys()
    4.  check_structure()
    5.  check_reaction_types()
    6.  check_reaction_consistency() 
    7.  check_species_eq_consistency() 
    8.  check_species_stoich_consistency()
    9.  check_species_in_db()
    10. check_input_file(rxns)
    11. parse_reaction_rate_params(reaction, convert_units) - Parse reaction coefficient params
    12. parse_stoich_coefs(string, species) - Parse stoichiometric coefficients
    13. get_weight(string, species) - Return total molecular weight for reactants/products of reactions
    
**\__init__**
    
    Sets class-wide parameters
    
**parse_species(rxns)**

    Parameters:
    rxns: XML File
    
    Returns:
    species: List of species
    
**check_keys()**

    Parameters:
    None
    
    Returns:
    ValueError if missing tags in XML file
    
**check_structure()**

    Parameters:
    None
    
    Returns:
    ValueError if structure of XML file not self-consistent
    
**check_reaction_types()**

    Parameters:
    None
    
    Returns:
    ValueError if reaction type not implemented in parser
    
**check_reaction_consistency()**

    Parameters:
    None
    
    Returns:
    ValueError if reaction type tag inconsistent with equation

**check_species_eq_consistency()**

    Parameters:
    None
    
    Returns:
    ValueError if species in phase array inconsistent with equations
    
**check_species_stoich_consistency()**

    Parameters:
    None
    
    Returns:
    None
    
**check_species_in_db()**

    Parameters:
    None
    
    Returns:
    ValueError if species in phase array inconsistent with reactants/products
    
**check_input_file(rxns, check_mass)**

    Parameters:
    rxns: XML File
    check_mass: Boolean
    
    Returns:
    ValueError if any of check_ methods fail

**parse_reaction_rate_params(reaction, convert_units)**

    Parameters:
    reaction: Input reaction
    convert_units: Boolean
    
    Returns:
    output: A list of parsed dictionaries for each reaction
    
**parse_stoich_coefs(string, species)**

    Parameters:
    string:
    species:
    
    Returns:
    coef_out:

**get_weight(string, species)**

    Parameters:
    string:
    species:
    
    Returns:
    total_weight:
 
---
    
## API Input Format
* Elementary reactions in XML-format
    * Currently, the following input format is accepted:

```xml
<ctml>
    <reactionData id="test_mechanism">
        <!-- reaction 01  -->
        <reaction reversible="yes" type="Elementary" id="reaction01">
            <equation>H + O2 [=] OH + O</equation>
            <rateCoeff>
                <Kooij>
                    <A units="cm3/mol/s">3.52e+16</A>
                    <b>-0.7</b>
                    <E units="kJ/mol">71.4</E>
                </Kooij>
            </rateCoeff>
            <reactants>H:1 O2:1</reactants>
            <products>OH:1 O:1</products>
        </reaction>
    </reactionData>
</ctml>
```

---

## Basic usage and examples

**Example 1:**

```python
# Import numpy and relevant flamespeed modules
import os
import numpy as np
from flamespeed import chemkin
from flamespeed.thermo import thermochem
from flamespeed.test import test_reaction_rate

# Set specie concentration vector
x = np.array([2.0, 1.0, 0.5, 1.0, 1.0])

# Initialize chemkin module with XML file
a = fs.ReactionRate()
a.read_XML('./flamespeed/data/rxns_hw5.xml')
print(a)

# Set temperature and get reaction rates
a.set_temp(750)
f = a.get_reaction_rate(x)
print(f)

--
Output:

Number_of_reactions:3                     
Number_of_species:5
Number_of_reversible_reactions:0
Reversible:False                     
Reaction_type:Elementary                     
Species_list:['H2', 'O2', 'OH', 'HO2', 'H2O']

Reaction rate:

[-3607077.87280406 -5613545.18362079  9220623.05642485  2006467.31081673
 -2006467.31081673]

```

**Example 2:**

```python
# Import numpy and relevant flamespeed modules
from flamespeed import chemkin

# Set specie concentration vector
x = [1.0, 1.0, 1.0, 1.0, 1.0]

# Initialize chemkin module with XML file
a = chemkin.ReactionRate()
a.read_XML('./flamespeed/data/rxns_hw5.xml')

# Set temperature and get reaction rates
a.set_temp(2500)
f = a.get_reaction_rate(x)
print(f)

--
Output:

Reaction rate:

[ -4.51065356e+08  -4.57236287e+08   9.08301644e+08   6.17093098e+06
  -6.17093098e+06]
```

**Example 3:**

```python
# Import numpy and relevant flamespeed modules
from flamespeed import chemkin

# Set specie concentration vector
x = [2.0, 1.0, 0.5, 1.0, 1.0, 1.5, 0.5, 1]

# Initialize chemkin module with XML file
a = chemkin.ReactionRate()
a.read_XML('./flamespeed/data/rxns_reversible_mixed.xml')
print(a)

# Set temperature and get reaction rates
a.set_temp(750)
f = a.get_reaction_rate(x)
print(f)

--
Output:

Number_of_reactions:11                   
Number_of_species:8
Number_of_reversible_reactions:7
Reversible:Some                     
Reaction_type:Elementary                     
Species_list:['H', 'O', 'OH', 'H2', 'H2O', 'O2', 'HO2', 'H2O2']

Reaction rate:

[-1.84263836e+14   3.18111565e+13   2.20832161e+14   4.07078985e+13
   3.44685565e+13  -1.19150183e+13  -7.63606019e+13  -5.52803168e+13]
```


**Example 4: Verify Integrity & Convert Units**

```python
# Import numpy and relevant flamespeed modules
from flamespeed import chemkin

# Set specie concentration vector
x = [0.5, 0, 0, 2, 0, 1]

# Initialize chemkin module with XML file
a = chemkin.ReactionRate()
a.read_XML('./flamespeed/data/rxns_units.xml', verify_integrity=True, convert_units=True)
print(a)

# Set temperature and get reaction rates
a.set_temp(900)
f = a.get_reaction_rate(x)
print(f)

--
Output:

Number_of_reactions:3                     
Number_of_species:6                     
Number_of_reversible_reactions:0                     
Reversible:No                     
Reaction_type:Elementary                     
Species_list:['H', 'O', 'OH', 'H2', 'H2O', 'O2']


Reaction rate:
[ -8.08297505e-14   8.08297505e-14   8.08297505e-14   0.00000000e+00
   0.00000000e+00  -8.08297505e-14]
```


**Example 5: (Parsing Hooked Reactions - ThreeBody, TroeThreeBody, Duplicate)**

```python
x = [0.5, 0, 0, 2, 0, 1]

# Import reaction data
a = chemkin.ReactionRate()
print ("[Three Body Parsed]")
a.read_XML('./flamespeed/data/rxn_ThreeBody.xml', verify_integrity=True, convert_units=True)
for idx, item in enumerate(a.k_params_nonelementary):
	print ("Reaction {}".format(str(idx)))
	for rxn in item:
		print (rxn)


print ("\n[Troe Three Body Parsed]")
a = chemkin.ReactionRate()
a.read_XML('./flamespeed/data/rxn_TroeFalloffThreeBody.xml', verify_integrity=True, convert_units=True)
for idx, item in enumerate(a.k_params_nonelementary):
	print ("Reaction {}".format(str(idx)))
	for rxn in item:
		print (rxn)

print ("\n[Duplicate Parsed]")
a = chemkin.ReactionRate()
a.read_XML('./flamespeed/data/rxn_Duplicate.xml', verify_integrity=True, convert_units=True)
for idx, item in enumerate(a.k_params_nonelementary):
	print ("Reaction {}".format(str(idx)))
	for rxn in item:
		print (rxn)

--
Output:

[Three Body Parsed]
Reaction 0
{'Type': 'Kooij', 'A': 38000000000.0, 'b': -2.0, 'E': 0.0, 'name': None}
{'Type': 'efficiencies', 'H2': '2.5', 'H2O': '12.0', 'default': '1.0'}
Reaction 1
{'Type': 'Kooij', 'A': 48000000000.0, 'b': -5.0, 'E': 4.1840000000000002, 'name': None}
{'Type': 'efficiencies', 'H2': '2.5', 'H2O': '12.0', 'default': '1.0'}

[Troe Three Body Parsed]
Reaction 0
{'Type': 'Kooij', 'A': 636600000.0, 'b': -1.72, 'E': 2195.7631999999999, 'name': 'k0'}
{'Type': 'Kooij', 'A': 1475000.0, 'b': 0.6, 'E': 0.0, 'name': None}
{'Type': 'Troe', 'alpha': 0.8, 't1': 1e+30, 't2': 1e+30, 't3': 1e+30}
{'Type': 'efficiencies', 'H2': '2.0', 'H2O': '11.0', 'O2': '0.78', 'default': '1.0'}

[Duplicate Parsed]
Reaction 0
{'Type': 'Arrhenius', 'A': 420000000.0, 'E': 50132.688000000002}
{'Type': 'Arrhenius', 'A': 130000.0, 'E': -6816.9912000000004}
```

---

## New Feature

#### 1. Motivation and feature description

The new features for the Flamespeed library adds flexibility by converting user inputs with relevant warnings if incomplete or non-standard input is provided. An array of validation checks are performed on the input data to ensure the input data is self-consistent, improving the robustness of the package.

**Flexibility**

* Additional usage flexibility is allowed for in several areas:

* Unit conversion:

         a. Parsed reaction coefficient data is converted to metric units before any 
           computations for a finite set of common conversions that might be required

* Incomplete Reaction Rate Parameters:

         a. Calling Arrhenius yet providing factor b in the XML file 
            results in flamespeed applying Modified Arrhenius 
        
* Three-Body Reactions:

         a. Parser extended to import parameters for three-body reactions
         b. Implemented hooks in code for further future development of three-body reactions

**Robustness and consistency**

* Additional robustness will be achieved following a two-pronged approach; namely checking user input consistency (i.e., in the form of XML files) as well as enforcing internal consistency throughout.

* User Input:
        
        a. Parsed chemical reaction equations are checked for conservation of mass
        b. Validation performed to ensure required data exists in thermo database for 
           input species

* Self-consistent reaction data in XML file, by performing the following integrity checks:

         a. XML file contains all the required tags and is self-consistent
         b. Reaction type tags are consistent with reaction type implied by equations
         c. Species in reaction equations correspond to the "species array" in XML file
         d. Species in product/reactant arrays correspond to "species array" in XML file
         
The primary goal of the flamespeed chemical kinetics library is for users to use the package for real-life chemical kinetics computations. As such, providing potential users with additional measures to assure both user input consistency and internal library consistency is deemed highly relevant and practical. Added flexibility in terms of reaction rate specification and three-body reaction hooks only add to the relevance of this library to practicioners.


#### 2. Implementation details

Another module (parser.py) that contains most of the features described above, was added to the library. Given the scope of the additional features it made sense to create a separate module, rather than including this in the main chemkin module. Changes were also made to the main module (chemkin.py) to integrate the new features.

An overview of the additional features added to each of the modules is included below. See Parser Module section above for a more detailed specification of the classes and methods included in the module.

**Module descriptions**

*parser.py*

* Contains all of the methods to check / validate self-consistency of the input data
* Includes methods to extract the list of species, reaction rate coefficient parameters and stoichiometric coefficients
* Checks reactions for conservation of mass and performs unit conversions if required

*chemkin.py*

* Contains the main method to read reaction data from the XML file (read_XML)
* Performing validation checks on the input file and converting units have been added as optional parameters to ensure consistency with other projects and previous versions of this project

**Unit Conversion**

A central part to the proposed feature is the possibility to convert a random set of units inherited from an `xml` input file into a set of consistent SI-units. A few points are worth highlighting below.

Where do the units occur: 
* Arrhenius prefactor "A"
    * Found in the `xml` input file
* Activation energy "E"
    * Found in the `xml` input file
    
What SI base units do conversions lead to:
* Meter (and its derivatives such as $m^3$, $m^6$, etc.)
* Joule
* Mol
* Seconds

What units are accounted for?
* A number of basic units are available as standard in the flamespeed package:
    * Length: ft, yd, cm, mm, etc.
    * Energy: cal, kcal, kJ, etc.
    * Time: minutes, etc.
* Users can expand the list of allowed units by simply expanding the `./data/units.csv` file with as many units as desired
    * In `./data/units.csv` users can also easily change the base units (i.e., it need not necessarily be those specified above)
    
**Conservation of mass**

The read_XML method that reads the reaction data from an XML file has an option to check conservation of mass for each of the reactions. In order to do this, the molecular weight of each specie inlcuded in the system is read from the `thermo` SQL database to compare the total mass of the reactants and products in each reaction. If conservation of mass is vioated for any reaction in the system an appropriate error message is displayed. This option can be turned off if there are valid species included in the reaction data, which is not included in the database.